# Conversation starter

This notebook goes over the basics of interacting with a Large Language Model using the Python library [LangChain](https://www.langchain.com/). Most of the example code here is adapted from the official [LangChain docs](https://python.langchain.com/docs/get_started/quickstart).

We will be using the OpenAI API to interact with GPT 3.5. This requires an API key and some additional steps for setup. Make sure you follow the instructions in the [README](../README.md) to get started. There is also additional information in the introductory [slide deck](../ppt/speedreading-with-llms.pdf).

<div class="alert alert-block alert-info">

**Note:** The OpenAI API is a paid service and running this notebook will cause the owner of the API key to get billed! GPT 3.5 costs only fractions of a cent per token (roughly 3/4 of a word), but you still should be mindful not to run unnecessary prompts, especially if they contain a lot of words.

</div>


## LangChain: A framework for LLM applications

_LangChain_ does not provide any Large Language Model by itself. Instead, it is a flexible framework that provides numerous components to build LLM-powered applications. Among other things, _LangChain_ offers a uniform interface to many Large Language Models, even though the LLMs themselves must be provided from elsewhere.

For example: If we want to use GPT 3.5 in a _LangChain_ application, we need to install OpenAI's Python library `openai`, as well. When we write the actual code, we only need to interact with _LangChain_ objects, but _LangChain_ calls the `openai` library "under-the-hood". This lets us use many LLMs in _LangChain_, so we can easily swap out the LLM in our application with hardly any changes to the code.

_LangChain_'s content can be roughly divided in two categories:

- **Components**: Useful abstractions for working with LLMs that help us quickly implement common problems
- **Chains**: A collection or sequence of components performing a particular task. We can use some pre-defined chains, for common tasks, implement our own, or do a mix of both.

This notebook will show you some examples from both of these categories.


## Loading our API key

At this point you should have set up a file named `secrets.env` with your OpenAI API key. We will now use a lightweight Python package called `dotenv` to read in this file and set its contents as environment variables:


## Accessing a Large Language Model

There are two types of language models, which in LangChain are called:

- LLMs: this is a language model which takes a single string as input and returns a single string
- ChatModels: this is a language model which takes a list of messages as input and returns a message

We use separate objects for each kind of model. Here, we will create an instance of each kind of model from the [GPT 3.5 family of models](https://platform.openai.com/docs/models/gpt-3-5):


And that's it! _LangChain_ takes care of everything else for us behind the scenes: Authenticating us using the API key, routing our requests to the right endpoints, retrieving and parsing the response. In the next section, we will take a look at how we can send a prompt to these models.


## Basic prompting using strings

The most straightforward way to send a prompt to the model is to use a string and `predict()` method, which is implemented by bothmodels:


Note that the chat model is essentially state-less and does not have any "memory" of the conversation in this case:


### Exercises

1. Try out a few different prompts. Notice the difference in the style of the response between the regular LLM and the chat model!

2. Try out the same prompt multiple times. Notice how the response changes!

<div class="alert alert-block alert-info">

Large Language Models have a parameter called `temperature` ranging from 0 to 1 that regulates this behavior: A higher value makes the output more different every time (like boiling water, always changing); a lower value makes the output change less (like frozen water, all molecules at rest). Even at a temperature of 0, however, the output for the same prompt is not guaranteed to be always the same.

By default, the `temperature` is set to `0.7`. There are two ways you can change the temperature:

```{python}
# During object creation
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)
# After object creation
llm.temperature = 0.1
```

</div>


## Prompting with `Message`s

An alternative to using a plain string to prompt a model is to use a `Message` object. Such objects contain the actual prompt, but they also contain a role descriptor in their `type` property:


You can prompt an LLM with a list of messages using the `predict_messages()` method. In that case, the model returns another `Message` object in response:


When using a chat model, you can now continue the conversation by just keeping a running list of the messages (both human and AI):


### Exercises

1. Start a new conversation by creating your own `HumanMessage`
2. Prompt the LLM with your message.
3. Keep track of the conversation by storing all messages in a list. Go back and forth with the model a couple of times!

<div class="alert alert-block alert-info">

Keeping track of the conversation manually can become rather tedious. _LangChain_ therefore offers extensive support to streamline this process (called [Memory](https://python.langchain.com/docs/modules/memory/)), but it is beyond the scope of this session.

</div>


## Prompt templates

Large Language Model applications often use very similar prompts over and over again with only minor changes to their content.

Continuing the previous example, we might want to ask for names for all of our different pets. In that case, we would ask the same question repeatedly, only changing the kind of pet we are asking about.

For such use cases, _LangChain_ offers the `PromptTemplate` object. This object allows us to create a template with a placeholder that we can replace with whatever string we like:


Now that we have such a template, we can iterate over a list of different kinds of pets and ask for name suggestions with very little code:


A more advanced version of the prompt template is based on `Message`s. In this `ChatPromptTemplate`, we can combine our regular prompt with a special system message that instructs the model how to behave in the conversation.


<div class="alert alert-block alert-info">

There is nothing truly special about the system message per se: It is also just a string. However, it is marked as "system", as opposed to "ai" or "human", and language models can be tuned to pay particular attention to this message to adapt their output style.

</div>


Let's use this template to ask for more pet names:


### Exercises

1. Define your own `PromptTemplate` and use it to prompt the `chat_model`.
2. Define your own `ChatPromptTemplate` and use it to prompt the `chat_model`.


## Building a Chain

You may have been wondering where the _Chain_ part in _LangChain_ is coming from. The code idea behind _LangChain_ is that you will eventually want to build an application that strings together multiple prompts, models, and/or other components. Such a combination of components is called a _Chain_.

For example, we have mostly used two components: prompt templates and LLMs. We have been using these components together the whole time, but we still referred to them separately: First we created the prompt from the template, then we plugged the prompt into the model.

Since we always use these components together, we might as well combine them in a chain. The most basic chain combines a prompt template and an LLM. It is called `LLMChain`:


Now we can run the chain by specifying all the required parameters for each component in the chain. In this case, only the prompt template has parameters:


### Exercises

1. Use the `ChatPromptTemplate` you defined above and create an `LLMChain` with it.
2. Run the chain for a couple of different parameter values in the template!

<div class="alert alert-block alert-info">

In addition to how we created the chain above, _LangChain_ also offers a clever way of composing chains called _LangChain Expression Language (LCEL)_. LCEL is beyond the scope of this session, but you can learn more about it in [the LangChain docs](https://python.langchain.com/docs/modules/chains/#lcel).

</div>


In the [next notebook](02-summarizing.ipynb), we will create a chain to do a slightly more meaningful task: Summarize text.

<table >
<tbody>
  <tr>
    <td style="padding:0px;border-width:0px;vertical-align:center">    
    Created by Simon Stone for Dartmouth College Library under <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons CC BY-NC 4.0 License</a>.<br>For questions, comments, or improvements, email <a href="mailto:researchdatahelp@groups.dartmouth.edu">Research Data Services</a>.
    </td>
    <td style="padding:0 0 0 1em;border-width:0px;vertical-align:center"><img alt="Creative Commons License" src="https://i.creativecommons.org/l/by/4.0/88x31.png"/></td>
  </tr>
</tbody>
</table>
